In [137]:
import datetime as dt
import requests
import requests_cache
requests_cache.install_cache("bigdave", expire_after=dt.timedelta(days=1)) 
from lxml import html
import re
from time import sleep

In [138]:
s = requests.Session()

Want to start at the home page (maybe don't cache this).  
Find the first page named 'DT \d{5}'  

Go to that page.  
Crawl the clues.  
Find the nav links, follow them until the next DT page is found.  
Repeat until no more nav links?  
Be polite.  


In [139]:
base_url = "http://bigdave44.com"

In [140]:
r = s.get(base_url)
r.raise_for_status()

In [141]:
tree = html.fromstring(r.content)
links = tree.xpath('//h2[@class="entry-title"]/a[@rel="bookmark"]/@href')
print(links)

['http://bigdave44.com/2009/01/28/welcome/', 'http://bigdave44.com/2023/03/22/toughie-3030/', 'http://bigdave44.com/2023/03/22/dt-30254/', 'http://bigdave44.com/2023/03/22/sunday-toughie-59-review/', 'http://bigdave44.com/2023/03/22/st-3203-full-review/', 'http://bigdave44.com/2023/03/21/rip-pam-morton-mrs-bd/', 'http://bigdave44.com/2023/03/21/167421/', 'http://bigdave44.com/2023/03/21/dt-30253/', 'http://bigdave44.com/2023/03/20/dt-30252/', 'http://bigdave44.com/2023/03/20/rookie-corner-467/', 'http://bigdave44.com/2023/03/19/sunday-toughie-60-hints/']


In [142]:
for l in links:
    if not 'dt-' in l:
        continue
    print(f"Getting {l}")
    break
r = s.get(l)
r.raise_for_status()

Getting http://bigdave44.com/2023/03/22/dt-30254/


In [143]:
def get_clues(l, content):
    clue_pairs = []
    tree = html.fromstring(content)
    if re.search('/dt-\d+/', l):
        clues = tree.xpath('//p')
        for _c in clues:
            c = re.sub(r'\s+', ' ', " ".join(_c.itertext()))
            c = re.sub(r'[—-]', '-', c)
            if not re.match(r'\d+[ad]', c):
                continue
            # print(c)
            clue, answer = None, None
            # Y’S 
            if (re.search(r'\d+[ad]\s+(.+\s\([\d,—-]+\))', c)):
                if (re.search(r'\([\d,—-]+\)\s+([A-Z\s—-]+)\s?[:—-]', c, re.I)):
                    clue = re.search(r'\d+[ad]\s+(.+\s\([\d,—-]+\))', c).groups()
                    answer = re.search(r'\([\d,—-]+\)\s+([A-Z’\'\s—-]+)\s?[:—-]', c, re.I).groups()
                    clue_pairs.append((clue[0].strip(), answer[0].strip())) 
                else:
                    print("Didn't find answer:", c)
                    
            else:
                # print("Didn't find clue:", c)
                ...
            
            # print (clue, answer)
    links = tree.xpath('//div [@class="nav-previous"]/a')
    
    # print(links[0].get('href'))
    return(clue_pairs, links[0].get('href'))
        

In [144]:
pairs = []
crawl=True
crawled_links = []
max_pages = 1800 # approx five years' worth
while crawl:
    if re.search('/dt-\d+/', l):
        crawled_links.append(l)
        print(l, f"({len(crawled_links)})")
    r = s.get(l)
    if not r.from_cache:
        print("Sleeping")
        sleep(1)
    r.raise_for_status()
    _pairs, l = get_clues(l, r.content)
    
    pairs.extend(_pairs)
    if not l:
        crawl = False
    if len(crawled_links) > max_pages:
        crawl=False

http://bigdave44.com/2023/03/22/dt-30254/ (1)
http://bigdave44.com/2023/03/21/dt-30253/ (2)
http://bigdave44.com/2023/03/20/dt-30252/ (3)
http://bigdave44.com/2023/03/17/dt-30250/ (4)
http://bigdave44.com/2023/03/15/dt-30248/ (5)
http://bigdave44.com/2023/03/14/dt-30247/ (6)
http://bigdave44.com/2023/03/10/dt-30244/ (7)
http://bigdave44.com/2023/03/08/dt-30242/ (8)
http://bigdave44.com/2023/03/07/dt-30241/ (9)
http://bigdave44.com/2023/03/06/dt-30240/ (10)
http://bigdave44.com/2023/03/03/dt-30238/ (11)
http://bigdave44.com/2023/03/01/dt-30236/ (12)
http://bigdave44.com/2023/02/28/dt-30235/ (13)
http://bigdave44.com/2023/02/27/dt-30234/ (14)
http://bigdave44.com/2023/02/24/dt-30232/ (15)
http://bigdave44.com/2023/02/22/dt-30230/ (16)
http://bigdave44.com/2023/02/21/dt-30229/ (17)
http://bigdave44.com/2023/02/20/dt-30228/ (18)
http://bigdave44.com/2023/02/17/dt-30226/ (19)
http://bigdave44.com/2023/02/15/dt-30224/ (20)
http://bigdave44.com/2023/02/14/dt-30223/ (21)
http://bigdave44.com/2

KeyboardInterrupt: 

In [ ]:
_str = " 17d Wisecrack from individual on ship (3-5) ONE-LINER — another word for individual and a large passenger ship"
re.search(r'\([\d,—-]+\)\s+([A-Z\s—-]+)\s?[:—-]', _str, re.I)


<re.Match object; span=(39, 56), match='(3-5) ONE-LINER —'>

In [146]:
len(pairs), pairs[0]

(2504,
 ('Giving affection, having a close relationship (4,2,5)', 'HAND IN GLOVE'))

In [147]:
with open('pairs.txt', 'w', encoding='utf-8') as outfile:
    for p in pairs:
        print("\t".join(p), file=outfile)

In [149]:
365*5

1825